In [ ]:
#installations
%pip install image | tail -n 1
%pip install -U "ibm_watsonx_ai>=1.1.14" | tail -n 1

In [ ]:
#imports
import requests
import base64
import getpass
import textwrap

from PIL import Image
from ibm_watsonx_ai import Credentials
from ibm_watsonx_ai.foundation_models import ModelInference

In [ ]:
WATSONX_EU_APIKEY = ("oj1dWP1BOzQ2V3rbTUXcSR947BOkPChxVGhEFSdEb0gc")

WATSONX_EU_PROJECT_ID = ("0024c858-01c7-4c29-9d40-1dd066cff517")

URL = "https://eu-gb.ml.cloud.ibm.com"

In [ ]:
credentials = Credentials(
    url=URL,
    api_key=WATSONX_EU_APIKEY
)

In [ ]:
import os, types
import pandas as pd
import ibm_boto3
from botocore.client import Config

def __iter__(self): return 0

# -------------------------------
# Load Data from IBM Cloud
# -------------------------------
def load_data_from_ibm():
    try:
        cos_client = ibm_boto3.client(
            service_name='s3',
            ibm_api_key_id='gbIhcR2aq4noHHkmCcUc5bv3ScAWGz08VLq0ziUU87UL',
            ibm_auth_endpoint="https://iam.cloud.ibm.com/identity/token",
            config=Config(signature_version='oauth'),
            endpoint_url='https://s3.direct.eu-gb.cloud-object-storage.appdomain.cloud'
        )

        bucket = 'learnmateai-donotdelete-pr-x8wp6f4miesbji'
        object_key = 'learnmate_keywords_expanded.csv'

        print("🔄 Loading course data from IBM Cloud Object Storage...")
        body = cos_client.get_object(Bucket=bucket, Key=object_key).get('Body')

        if not hasattr(body, "__iter__"):
            body.__iter__ = types.MethodType(__iter__, body)

        df = pd.read_csv(body)

        if df is None or df.empty:
            print("❌ Loaded an empty dataset. Please check the file.")
            exit()

        return df

    except Exception as e:
        print(f"❌ Failed to load dataset: {e}")
        exit()


In [ ]:

# -------------------------------
# User Input Function
# -------------------------------
def get_user_input():
    print("\n👋 Welcome to LearnMate!")
    print("Type 'stop' or 'bye' anytime to exit.\n")
    interest = input("🎯 What are you interested in learning? (e.g., UI/UX Design, Cybersecurity): ").strip()

    if interest.lower() in ['stop', 'bye']:
        return interest, None

    level = input("⭐ Select your skill level (Beginner / Intermediate / Advanced): ").strip().capitalize()
    return interest, level


# -------------------------------
# Suggest Closest Matches
# -------------------------------
def suggest_closest_match(df, interest):
    categories = df["Course Name"].unique()
    matches = [cat for cat in categories if interest.lower() in cat.lower()]
    if not matches:
        matches = categories[:3]
    print("\n🔍 Did you mean one of these?")
    for m in matches:
        print(f"👉 {m}")


# -------------------------------
# Generate and Display Roadmap
# -------------------------------
def generate_roadmap(df, interest, level):
    filtered = df[df["Course Name"].str.contains(interest, case=False, na=False)].copy()
    filtered.reset_index(drop=True, inplace=True)

    if filtered.empty:
        print(f"\n❗ Sorry, we couldn't find results for '{interest}'.")
        suggest_closest_match(df, interest)
        return

    print(f"\n📘 Your {level} Roadmap for: {interest}\n")
    for i, row in filtered.iterrows():
        print(f"{i+1}. {row['Keyword']} — {row['Explanation']}")


# -------------------------------
# Optional Console Clear
# -------------------------------
def clear_console():
    os.system('cls' if os.name == 'nt' else 'clear')


# -------------------------------
# Main Program Loop
# -------------------------------
def main():
    df = load_data_from_ibm()
    df = load_data_from_ibm()
    if df is None:
        print("❌ Dataset could not be loaded. Exiting.")
        return


    while True:
        interest, level = get_user_input()

        if interest is None or interest.lower() in ['stop', 'bye']:
            print("\n👋 Thank you for using LearnMate. Goodbye!")
            break

        if not level:
            print("⚠️ Skill level not provided. Please try again.")
            continue

        generate_roadmap(df, interest, level)

        input("\n🔁 Press Enter to explore another track...")
        clear_console()



